In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# for saving the pipeline
import joblib

# from Scikit-learn
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, Binarizer
from sklearn.ensemble._forest import RandomForestRegressor

# from feature-engine
from feature_engine.imputation import (
    AddMissingIndicator,
    MeanMedianImputer,
    CategoricalImputer,
)

from feature_engine.encoding import (
    RareLabelEncoder,
    OrdinalEncoder,
    OneHotEncoder
)

from feature_engine.transformation import (
    LogTransformer,
    YeoJohnsonTransformer,
)

from feature_engine.selection import DropFeatures
from feature_engine.wrappers import SklearnTransformerWrapper

import Libs.preprocessors as pp

# Feature Engineering

In [2]:
df=pd.read_csv('data/train.csv')
features=pd.read_csv('output/selected_features.csv')

In [3]:
df['running']=df['running'].apply(lambda x: float(x.replace('km','')) if x[-2:]=='km' else float(x.replace('miles',''))*1.609344)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['price','wheel'], axis=1), # predictive variables
    df['price'], # target
    test_size=0.1, # portion of dataset to allocate to test set
    random_state=0, # we are setting the seed here
)

X_train.shape, X_test.shape

((1477, 8), (165, 8))

In [5]:
ONEHOTENCODER_VARS = ['model', 'motor_type','color','type']

TEMPORAL_VARS = ['year']

NUMERICALS_YEO_VARS = ["running", "motor_volume"]

QUAL_VARS = [
    'status']

qual_mappings = {'excellent': 3, 'good':2, 'crashed': 0, 'normal': 1, 'new': 4}

In [6]:
pipeline = Pipeline([

    # == TEMPORAL VARIABLES ====
    ('elapsed_time', pp.TemporalVariableTransformer(
        variables=TEMPORAL_VARS)),

    # ===================== VARIABLE TRANSFORMATION ======================

    ('yeojohnson', YeoJohnsonTransformer(variables=NUMERICALS_YEO_VARS)),


    # =========================== mappers ===============================
    ('mapper_qual', pp.Mapper(
        variables=QUAL_VARS, mappings=qual_mappings)),

    # == CATEGORICAL ENCODING
    ('one_hot_encoder', pp.CategoricalOneHotEncoder(
        variables=ONEHOTENCODER_VARS
    )),

    ('column_selector', pp.ColumnSelector(columns=features['0'].values)),

    ('ColumnsTRansformOHE',pp.MotorTypeTransformer()),

    ('scaler', MinMaxScaler()),
    ('selector', RandomForestRegressor(n_estimators=100,random_state=123)),
])

In [7]:
# train the pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('elapsed_time',
                 TemporalVariableTransformer(variables=['year'])),
                ('yeojohnson',
                 YeoJohnsonTransformer(variables=['running', 'motor_volume'])),
                ('mapper_qual',
                 Mapper(mappings={'crashed': 0, 'excellent': 3, 'good': 2,
                                  'new': 4, 'normal': 1},
                        variables=['status'])),
                ('one_hot_encoder',
                 CategoricalOneHotEncoder(variables=['model', 'motor_type',
                                                     'color',...
       'color_clove', 'color_golden', 'color_gray', 'color_green',
       'color_orange', 'color_other', 'color_pink', 'color_purple',
       'color_red', 'color_silver', 'color_skyblue', 'color_white',
       'type_Coupe', 'type_Universal', 'type_hatchback', 'type_pickup',
       'type_sedan', 'type_suv'], dtype=object))),
                ('ColumnsTRansformOHE', MotorTypeTransformer()),
                ('scaler', MinMaxScaler()),
                ('selector', RandomForestRegressor(random_state=123))])

In [8]:
# evaluate the model:
# ====================

# make predictions for train set
pred = pipeline.predict(X_train)

# determine mse, rmse and r2
print('train mse: {}'.format(int(
    mean_squared_error(y_train, pred))))
print('train rmse: {}'.format(int(
    mean_squared_error(y_train, pred, squared=False))))
print('train r2: {}'.format(
    r2_score(y_train, pred)))
print()

# make predictions for test set
pred = pipeline.predict(X_test)

# determine mse, rmse and r2
print('test mse: {}'.format(int(
    mean_squared_error(y_test, pred))))
print('test rmse: {}'.format(int(
    mean_squared_error(y_test, pred, squared=False))))
print('test r2: {}'.format(
    r2_score(y_test, pred)))
print()

print('Average Car price: ', int(y_train.median()))

train mse: 2221405
train rmse: 1490
train r2: 0.9578152108422667

test mse: 8477308
test rmse: 2911
test r2: 0.791467404634859

Average house price:  15700


c:\Users\56953\Proyectos Python\Datapath ML\Modulo 2\ModeloMLDesp\venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\56953\Proyectos Python\Datapath ML\Modulo 2\ModeloMLDesp\venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [9]:
joblib.dump(pipeline, './Output/pipeline.joblib')

['./Output/pipeline.joblib']